In [1]:
from predictLSTM import predictLSTM
import tool
from sweDataset import sweDataset
import torch
import pandas as pd
import numpy as np

In [2]:
folder_path = '/Users/yz6622/Desktop/IRP/dataset/swiss_station/Preprocessed_data/'

In [3]:
# Define variables
train_file_path = folder_path + 'train.csv'
test_file_path = folder_path + 'test.csv'
var = ['HS']
ts = 30

# read data
df = pd.read_csv(train_file_path)
df_test = pd.read_csv(test_file_path)

In [20]:
df = tool.inverse_scale(df)
df_test = tool.inverse_scale(df_test)

In [4]:
df_norway = df[df['loc'].str.startswith('norway')]
norway_df_test = df_test[df_test['loc'].str.startswith('norway')]
df_canada = df[df['loc'].str.startswith('canada')]
canada_df_test = df_test[df_test['loc'].str.startswith('canada')]
df_swi = df[(df['loc'].str.startswith('gps')) | (df['loc']=='wfj')]
swi_df_test = df[df['loc']=='klo']

In [22]:
scaler_y_dict={}
for countries, sub_df, sub_df_test in zip([1,2,3], [df_norway,df_canada,df_swi], [norway_df_test,canada_df_test,swi_df_test]):
    # Standardize specific columns
    sub_df['temperature'], scaler_temp = tool.standardise_data(np.array(sub_df['temperature']).reshape(-1,1), 'train')
    sub_df[['HS','precipitation','snowfall','solar_radiation','rain','month']], scaler_minmax = tool.minmax_data(sub_df[['HS','precipitation','snowfall','solar_radiation','rain','month']], 'train')
    sub_df['station_SWE'], scaler_y = tool.minmax_data(np.array(sub_df['station_SWE']).reshape(-1,1), 'train')
    # Save the scaler to dictionary
    scaler_y_dict[countries] = scaler_y
    
    # Apply the same scaling to test set
    sub_df_test['temperature'], _ = tool.standardise_data(np.array(sub_df_test['temperature']).reshape(-1,1), 'test', scaler_temp)
    sub_df_test[['HS','precipitation','snowfall','solar_radiation','rain','month']], _ = tool.minmax_data(sub_df_test[['HS','precipitation','snowfall','solar_radiation','rain','month']], 'test', scaler_minmax)
    sub_df_test['station_SWE'], _ = tool.minmax_data(np.array(sub_df_test['station_SWE']).reshape(-1,1), 'test', scaler_y)

/var/folders/5m/bhjjh_l15ng7p2l7swm_bkzh0000gp/T/ipykernel_50014/3233331708.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['temperature'], scaler_temp = tool.standardise_data(np.array(sub_df['temperature']).reshape(-1,1), 'train')
/var/folders/5m/bhjjh_l15ng7p2l7swm_bkzh0000gp/T/ipykernel_50014/3233331708.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df[['HS','precipitation','snowfall','solar_radiation','rain','month']], scaler_minmax = tool.minmax_data(sub_df[['HS','precipitation','sn

In [23]:
input_data, target_data = tool.rebuild_data(canada_df_test, ['HS'], ts=30)

/Users/yz6622/Desktop/IRP/LSTM/tool.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loc_df['date'] = pd.to_datetime(loc_df['date'])
/Users/yz6622/Desktop/IRP/LSTM/tool.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loc_df['date'] = pd.to_datetime(loc_df['date'])
/Users/yz6622/Desktop/IRP/LSTM/tool.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https

In [24]:
def rename_keys(model_path):
    # Load the original state dict
    state_dict = torch.load(model_path)

    # Create a new dictionary with the renamed keys
    new_state_dict = {}
    for key, value in state_dict.items():
        # Rename the keys here
        new_key = key.replace("lstm1", "lstm_layers.0").replace("lstm2", "lstm_layers.1")
        new_state_dict[new_key] = value

    # Save the modified state dict
    torch.save(new_state_dict, model_path)

In [9]:
rename_keys("/Users/yz6622/Desktop/IRP/models/global_model.pth")
rename_keys("/Users/yz6622/Desktop/IRP/models/model_norway.pth")
rename_keys("/Users/yz6622/Desktop/IRP/models/model_switzerland.pth")
rename_keys("/Users/yz6622/Desktop/IRP/models/model_canada.pth")


In [10]:
# Create ensemble models
ensemble_model = predictLSTM("/Users/yz6622/Desktop/IRP/models/model_canada.pth")

In [11]:
# Add models for different counties
for country in ['canada', 'switzerland', 'norway']:
    model_path = f'/Users/yz6622/Desktop/IRP/models/model_{country}.pth'
    ensemble_model.add_country_model(country, model_path)


In [5]:
dataset=sweDataset(df=df_canada, df_test=canada_df_test, var=var, ts=ts)

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [16]:
country_prediction = ensemble_model.predict_by_country('canada', input_data)

In [ ]:
# Add models for different snow class
for snow_class in [1, 3, 6]:
    model_path = f'/Users/yz6622/Desktop/IRP/models/model_sc_{snow_class}.pth' 
    ensemble_model.add_snowclass_model(snow_class, model_path)